In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader

import os

base_dir = os.getcwd() 
file_path = os.path.join(base_dir, "..", "CorrectionDispersion", "dataset", "nps_simulated_dataset_gaussiano_2025-08-08.csv")
dataset = pd.read_csv(file_path)

print("Number of rows: ", dataset.shape[0])
print("Number of columns: ", dataset.shape[1])
dataset.head(20)

Number of rows:  1000
Number of columns:  21


,simulation_id,sensor_id,sensor_x,sensor_y,sensor_noise,sensor_height,days,RH,humidify,wind_type,...,wind_dir,stability_profile,stability_value,aerosol_type,source_x,source_y,source_h,emission_rate,real_concentration,contratio_series
0,0,0,336.0,116.0,0.0001,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"2.5358001309986458e-11,1.872893329031193e-13,0..."
1,0,1,295.0,103.0,0.0002,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"0.0,1.7341939398685742e-10,0.0,0.0,0.0,0.0,0.0..."
2,0,2,175.0,298.0,0.0003,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"0.0,2.4068992219435856e-10,0.0,2.1037543010111..."
3,0,3,235.0,259.0,0.0004,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"0.0,0.0,0.0,4.1187540596887963e-10,2.279009879..."
4,0,4,222.0,430.0,0.0001,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"1.1456366472159734e-10,5.838943145112818e-11,0..."
5,0,5,457.0,251.0,0.0001,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"3.630566618251091e-11,9.539856806229742e-11,2...."
6,0,6,45.0,496.0,0.0005,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"0.0,1.2220804000078467e-09,0.0,3.7321703153347..."
7,0,7,147.0,453.0,0.0001,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"8.014782738402648e-11,0.0,0.0,0.0,0.0,1.745078..."
8,0,8,332.0,454.0,0.0001,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"8.551388405286455e-11,7.694261053677018e-11,0...."
9,0,9,228.0,79.0,0.0001,2.0,10,0.5,True,FLUCTUATING,...,"105.89181694366846,76.84505725432611,117.24579...",CONSTANT,PasquillGiffordStability.NEUTRAL,OTHER_COMPOUNDS,93.0,431.0,3.60,0.0085,sim_0_conc_real_2025-08-08.npy,"1.8390768075747678e-10,0.0,6.838639582178082e-..."


In [117]:
dataset.columns

Index(['simulation_id', 'sensor_id', 'sensor_x', 'sensor_y', 'sensor_noise',
       'sensor_height', 'days', 'RH', 'humidify', 'wind_type', 'wind_speed',
       'wind_dir', 'stability_profile', 'stability_value', 'aerosol_type',
       'source_x', 'source_y', 'source_h', 'emission_rate',
       'real_concentration', 'contratio_series'],
      dtype='object')

In [118]:
datset_copy= dataset.copy()

In [119]:
def convert_string_to_array(s):
    if isinstance(s, str):
        s = s.replace('[', '').replace(']', '')
        return np.fromstring(s, sep=',')
    return s

datset_copy["contratio_series"] = datset_copy["contratio_series"].apply(convert_string_to_array) #type: ignore
datset_copy["wind_dir"] = datset_copy["wind_dir"].apply(convert_string_to_array) #type: ignore

In [120]:
datset_copy["wind_dir"]

0      [105.89181694366846, 76.84505725432611, 117.24...
1      [105.89181694366846, 76.84505725432611, 117.24...
2      [105.89181694366846, 76.84505725432611, 117.24...
3      [105.89181694366846, 76.84505725432611, 117.24...
4      [105.89181694366846, 76.84505725432611, 117.24...
                             ...                        
995    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
996    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
997    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
998    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
999    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: wind_dir, Length: 1000, dtype: object

In [121]:
wind_type_map = {
    "CONSTANT": 1,
    "FLUCTUATING": 2,
    "PREVAILING": 3
}

pg_stability_map = {
    "PasquillGiffordStability.VERY_UNSTABLE": 1,
    "PasquillGiffordStability.MODERATELY_UNSTABLE": 2,
    "PasquillGiffordStability.SLIGHTLY_UNSTABLE": 3,
    "PasquillGiffordStability.NEUTRAL": 4,
    "PasquillGiffordStability.MODERATELY_STABLE": 5,
    "PasquillGiffordStability.VERY_STABLE": 6
}

nps_type_map = {
    "CANNABINOID_ANALOGUES": 0,
    "CATHINONE_ANALOGUES": 1,
    "PHENETHYLAMINE_ANALOGUES": 2,
    "PIPERAZINE_ANALOGUES": 3,
    "TRYPTAMINE_ANALOGUES": 4,
    "FENTANYL_ANALOGUES": 5,
    "OTHER_COMPOUNDS": 6
}


datset_copy['wind_type'] = datset_copy['wind_type'].astype(str)
datset_copy['stability_value'] = datset_copy['stability_value'].astype(str)
datset_copy['aerosol_type'] = datset_copy['aerosol_type'].astype(str)

datset_copy['wind_type'] = datset_copy['wind_type'].map(wind_type_map)
datset_copy['stability_value'] = datset_copy['stability_value'].map(pg_stability_map)
datset_copy['aerosol_type'] = datset_copy['aerosol_type'].map(nps_type_map)

datset_copy[["wind_type", "stability_value", "aerosol_type"]].head()

,wind_type,stability_value,aerosol_type
0,2,4,6
1,2,4,6
2,2,4,6
3,2,4,6
4,2,4,6


In [122]:
datset_copy.drop(columns=["humidify", "stability_profile", "sensor_height"], inplace=True)

In [123]:
datset_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   simulation_id       1000 non-null   int64  
 1   sensor_id           1000 non-null   int64  
 2   sensor_x            1000 non-null   float64
 3   sensor_y            1000 non-null   float64
 4   sensor_noise        1000 non-null   float64
 5   days                1000 non-null   int64  
 6   RH                  1000 non-null   float64
 7   wind_type           1000 non-null   int64  
 8   wind_speed          1000 non-null   float64
 9   wind_dir            1000 non-null   object 
 10  stability_value     1000 non-null   int64  
 11  aerosol_type        1000 non-null   int64  
 12  source_x            1000 non-null   float64
 13  source_y            1000 non-null   float64
 14  source_h            1000 non-null   float64
 15  emission_rate       1000 non-null   float64
 16  real_co

In [124]:
real_contration= datset_copy['real_concentration']
#datset_copy.drop(columns=["real_concentration"], inplace=True)

In [125]:
datset_copy.head()

,simulation_id,sensor_id,sensor_x,sensor_y,sensor_noise,days,RH,wind_type,wind_speed,wind_dir,stability_value,aerosol_type,source_x,source_y,source_h,emission_rate,real_concentration,contratio_series
0,0,0,336.0,116.0,0.0001,10,0.5,2,7.16,"[105.89181694366846, 76.84505725432611, 117.24...",4,6,93.0,431.0,3.6,0.0085,sim_0_conc_real_2025-08-08.npy,"[2.5358001309986458e-11, 1.872893329031193e-13..."
1,0,1,295.0,103.0,0.0002,10,0.5,2,7.16,"[105.89181694366846, 76.84505725432611, 117.24...",4,6,93.0,431.0,3.6,0.0085,sim_0_conc_real_2025-08-08.npy,"[0.0, 1.7341939398685742e-10, 0.0, 0.0, 0.0, 0..."
2,0,2,175.0,298.0,0.0003,10,0.5,2,7.16,"[105.89181694366846, 76.84505725432611, 117.24...",4,6,93.0,431.0,3.6,0.0085,sim_0_conc_real_2025-08-08.npy,"[0.0, 2.4068992219435856e-10, 0.0, 2.103754301..."
3,0,3,235.0,259.0,0.0004,10,0.5,2,7.16,"[105.89181694366846, 76.84505725432611, 117.24...",4,6,93.0,431.0,3.6,0.0085,sim_0_conc_real_2025-08-08.npy,"[0.0, 0.0, 0.0, 4.1187540596887963e-10, 2.2790..."
4,0,4,222.0,430.0,0.0001,10,0.5,2,7.16,"[105.89181694366846, 76.84505725432611, 117.24...",4,6,93.0,431.0,3.6,0.0085,sim_0_conc_real_2025-08-08.npy,"[1.1456366472159734e-10, 5.838943145112818e-11..."


In [126]:
datset_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   simulation_id       1000 non-null   int64  
 1   sensor_id           1000 non-null   int64  
 2   sensor_x            1000 non-null   float64
 3   sensor_y            1000 non-null   float64
 4   sensor_noise        1000 non-null   float64
 5   days                1000 non-null   int64  
 6   RH                  1000 non-null   float64
 7   wind_type           1000 non-null   int64  
 8   wind_speed          1000 non-null   float64
 9   wind_dir            1000 non-null   object 
 10  stability_value     1000 non-null   int64  
 11  aerosol_type        1000 non-null   int64  
 12  source_x            1000 non-null   float64
 13  source_y            1000 non-null   float64
 14  source_h            1000 non-null   float64
 15  emission_rate       1000 non-null   float64
 16  real_co

In [ ]:
def build_full_sequences(df, feature_cols, target_cols):
    X, y = [], []
    grouped = df.groupby('simulation_id')

    for _, group in grouped:
        features_seq = []

        data=[]
        # Estraggo features in ordine
        feature_list = []
        for col in feature_cols:
            if col in ["contratio_series", "wind_dir"]:
                # Aggiungo tutti i valori del vettore
                arr = np.stack(data[col].to_numpy())  # (num_sensori, len_vettore)
                feature_list.append(arr)
            else:
                vals = data[col].to_numpy(dtype=np.float32).reshape(-1, 1)  # (num_sensori, 1)
                feature_list.append(vals)

            # Concateno tutte le feature di quel giorno (num_sensori, tot_feature) → flatten
            day_features = np.concatenate(feature_list, axis=1).flatten()
            features_seq.append(day_features)

        X.append(np.array(features_seq, dtype=np.float32))
        y.append(group[target_cols].iloc[0].to_numpy(dtype=np.float32))

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)


feature_columns = ['contratio_series', 'sensor_x', 'sensor_y',
                   'sensor_noise', 'RH', 'wind_type', 'wind_speed',
                   'wind_dir', 'stability_value', 'aerosol_type',
                   'emission_rate']

target_columns = ['source_x', 'source_y', 'source_h']

X, y = build_full_sequences(datset_copy, feature_columns, target_columns)
print("X shape:", X.shape)  # (num_sim, 10, num_sensori*feature_per_sensore)
print("y shape:", y.shape)  # (num_sim, 3)


X shape: (100, 1, 4890)
y shape: (100, 3)


In [131]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset_torch = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset_torch, batch_size=50, shuffle=True)

# %% ---- Modello LSTM
class SourcePredictor(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SourcePredictor, self).__init__()
        self.lstm = torch.nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)       # (batch, seq_len, hidden)
        last_hidden = lstm_out[:, -1, :] # ultimo timestep
        return self.fc(last_hidden)

input_size = X.shape[2]
model = SourcePredictor(input_size, hidden_size=64, num_layers=1, output_size=3)

# %% ---- Loss e ottimizzatore
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# %% ---- Training loop
for epoch in range(50):
    model.train()
    total_loss = 0
    for xb, yb in loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")


Epoch 1, Loss: 57357.2715
Epoch 2, Loss: 57306.5625
Epoch 3, Loss: 57279.7227
Epoch 4, Loss: 57261.4922
Epoch 5, Loss: 57243.0508
Epoch 6, Loss: 57228.1934
Epoch 7, Loss: 57213.5820
Epoch 8, Loss: 57198.7656
Epoch 9, Loss: 57185.7910
Epoch 10, Loss: 57172.0234
Epoch 11, Loss: 57161.1406
Epoch 12, Loss: 57147.6660
Epoch 13, Loss: 57134.5684
Epoch 14, Loss: 57121.1504
Epoch 15, Loss: 57101.8965
Epoch 16, Loss: 57084.6230
Epoch 17, Loss: 57070.3203
Epoch 18, Loss: 57056.2969
Epoch 19, Loss: 57040.8516
Epoch 20, Loss: 57027.1680
Epoch 21, Loss: 57009.1992
Epoch 22, Loss: 56993.5918
Epoch 23, Loss: 56979.0781
Epoch 24, Loss: 56965.1191
Epoch 25, Loss: 56951.1602
Epoch 26, Loss: 56936.2227
Epoch 27, Loss: 56922.0859
Epoch 28, Loss: 56908.3496
Epoch 29, Loss: 56895.0684
Epoch 30, Loss: 56880.2852
Epoch 31, Loss: 56866.7617
Epoch 32, Loss: 56852.9648
Epoch 33, Loss: 56840.7480
Epoch 34, Loss: 56827.3906
Epoch 35, Loss: 56814.5723
Epoch 36, Loss: 56801.7832
Epoch 37, Loss: 56788.6523
Epoch 38, 